In [ ]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

In [ ]:
from google.colab import userdata
import os

# Retrieve AWS credentials from Colab Secrets
aws_access_key = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_key = userdata.get('AWS_SECRET_ACCESS_KEY')
mlflow_uri = userdata.get('MLFLOW_TRACKING_URI')

# Set as environment variables (for boto3 and AWS CLI)
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_key

In [ ]:
import mlflow
mlflow.set_tracking_uri(mlflow_uri)
mlflow.set_experiment("Exp 5 - ML Algos with hp Tuning")

2025/07/25 01:45:14 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with hp Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sumanthdatahub/mlflow-artifacts/805851809516210167', creation_time=1753407914079, experiment_id='805851809516210167', last_update_time=1753407914079, lifecycle_stage='active', name='Exp 5 - ML Algos with hp Tuning', tags={}>

In [ ]:
import optuna
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
# Read Preprocesssing Data
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36710, 6)

In [ ]:
# XGBoost Doesn't support -ve values so we transform
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Remove rows where target labels (Category) are NaN
df = df.dropna(subset=['category'])
ngram_range = (1,3)
max_features = 1000

# Vectorization using TF_IDF, fit on training data only
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_vec, y_train_vec = smote.fit_resample(X_train_vec, y_train)

In [ ]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run() as run:
        # Set Tags
        tags = {
            "mlflow.runName": f"{model_name}_SMOTE_TFIDF_Trigrams",
            "experiment_type": "Algorithm Comparision",
            "algo_name": model_name,
        }
        mlflow.set_tags(tags)

        # Train Model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log Metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log Classification Report
        report = classification_report(y_test, y_pred, output_dict=True)

        for label, metrics in report.items():
          if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)
        mlflow.log_model(model, f"{model_name}_model")

# Optuna Objective Function for XGBoost
def objective_xgboost(trial):
  n_estimators = trial.suggest_int("n_estimators", 50, 300)
  learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
  max_depth = trial.suggest_int("max_depth", 3, 10)

  model = XGBClassifier(
      n_estimators=n_estimators,
      learning_rate=learning_rate,
      max_depth=max_depth,
      random_state=42
  )
  return accuracy_score(y_test, model.fit(X_train_vec, y_train_vec).predict(X_test_vec))

# Run Optuna For XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the Best params and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params["n_estimators"],
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        random_state=42
    )

    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train_vec, y_test)

In [ ]:
# Run Optuna Experiment
run_optuna_experiment()

[I 2025-07-25 01:45:43,557] A new study created in memory with name: no-name-8da5b18c-e05e-4fce-91cb-2a8a64595e12
[I 2025-07-25 01:48:13,643] Trial 0 finished with value: 0.5363661127758104 and parameters: {'n_estimators': 245, 'learning_rate': 0.00017456858083376878, 'max_depth': 5}. Best is trial 0 with value: 0.5363661127758104.
[I 2025-07-25 01:48:44,989] Trial 1 finished with value: 0.5435848542631435 and parameters: {'n_estimators': 52, 'learning_rate': 0.002989125533083698, 'max_depth': 5}. Best is trial 1 with value: 0.5435848542631435.
[I 2025-07-25 01:58:53,339] Trial 2 finished with value: 0.6491419231816944 and parameters: {'n_estimators': 273, 'learning_rate': 0.003254808364476957, 'max_depth': 10}. Best is trial 2 with value: 0.6491419231816944.
[I 2025-07-25 01:59:46,914] Trial 3 finished with value: 0.545491691637156 and parameters: {'n_estimators': 139, 'learning_rate': 0.0020379053347280994, 'max_depth': 4}. Best is trial 2 with value: 0.6491419231816944.
[I 2025-07-2